In [36]:
import os
import folium
from folium.plugins import Draw
import geopandas as gpd
import meshio
import pandas as pd
import pyproj as proj
from shapely.geometry import Point, LineString, Polygon



In [37]:
## Draw map with geojson attributes
m = folium.Map()
draw = Draw(export=True)
draw.add_to(m)
m

In [38]:
gjsn_file=gpd.read_file(filename='Downloads/data.geojson') # Read geoJSON file
series = gpd.GeoSeries(gjsn_file) # To GeoSeries
"""
type(series[0])
len(series[0].coords[:])
for i in range(len(series[0].coords[:])):
    print(series[0].coords[i])
"""  
p=[series[0].coords[i] for i in range(len(series[0].coords[:]))] # LineString to points

In [39]:
## Import the vtk file and read it using the meshio
vtk_file=meshio.read('/home/su533162/Project/pyert/surveys/MEMORIS_crosshole/inversions/F10F11F12F13F14F15F16_3D_20180315_inv_recipe/F10F11F12F13F14F15F16_3D_20180315_inv_recipe_res_vtk.vtk')

In [40]:
## From the vtk file we extract all the coordinates 
vtk_coords=gpd.GeoDataFrame(vtk_file[0])
vtk_coords=pd.DataFrame([vtk_coords[0]+152000,vtk_coords[1]+122000]).T # Add the correct values to correct the coordinates 
#vtk_coords

In [41]:
## Transform vtk coordinates to WGS84
crs_wgs = proj.Proj(init='epsg:4326')
crs_lb72 = proj.Proj(init='epsg:31370') 
x2,y2=proj.transform(crs_lb72,crs_wgs,vtk_coords[0][0],vtk_coords[1][0]) # Random (first) point for transformation
## Tranformation of all points (not sure if needed)
vtk_wgs84=[proj.transform(crs_lb72,crs_wgs,vtk_coords[0][i],vtk_coords[1][i]) for i in range(len(vtk_coords))]
vtk_wgs84=[vtk_wgs84[i][::-1] for i in range(len(vtk_wgs84))] # Reverse order of points (lat-long) 



In [42]:
## Find center of line for popup image of the vtk slice

p1=p[0][::-1]
p2=p[1][::-1]
line=LineString([Point(p1[0],p1[1]),Point(p2[0],p2[1])])
line.centroid.coords[0]
print('p1=',p1)
print('p2=',p2)
print('center=',line.centroid.coords[0])


p1= (40.5245, 22.947006)
p2= (40.516866, 22.986488)
center= (40.520683000000005, 22.966746999999998)


In [43]:
## Plot to map the line, the center with marker and the vtk slice as image popup
m=folium.Map(location=line.centroid.coords[0], tiles="openstreetmap", zoom_start=10) # lat-long
#folium.Marker(p2).add_to(m)
#folium.Marker(p1).add_to(m)
folium.Marker(line.centroid.coords[0]).add_to(m)
folium.PolyLine([p2,p1]).add_to(m)
m